In [1]:
import sys
print (sys.version)

2.7.12 |Anaconda 4.1.1 (64-bit)| (default, Jun 29 2016, 11:07:13) [MSC v.1500 64 bit (AMD64)]


In [2]:
from __future__ import division
%matplotlib inline
import re
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pymysql
import scipy.stats
from sklearn import linear_model
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
import os
import random
import itertools
import warnings
import shutil
import pandas as pd
warnings.filterwarnings("ignore")
from scipy.sparse import coo_matrix, vstack,csr_matrix

In [3]:
#1. Tokenization
#2. Lematization
#3. Lower case

In [4]:
import nltk.tokenize
import nltk.data
from nltk.tokenize import StanfordTokenizer
from nltk.tokenize import word_tokenize
#sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

In [5]:
data=pd.DataFrame(columns=["dk_ref","fname","decision_fk","lawyer_name","text_data"])
db = pymysql.connect(host="localhost", port=3320, user="vicky", passwd="vicky9790851962", db="fulltextdarts", charset="utf8")
cursor = db.cursor()

In [6]:
# query="SELECT dk.reference, filename, decision_fk,euipodl.lawyer_name,textdata FROM fulltextdarts.plaintext pt JOIN darts.decision d ON pt.decision_fk=d.id JOIN darts.court c ON court_fk=c.id JOIN darts.docket dk ON d.docket_fk=dk.id JOIN judges.euipo_docket_lawyer euipodl ON euipodl.opp_number=dk.reference WHERE c.id=1121 AND pt.language='en' AND dk.reference REGEXP '[0-9]{9}';"
# cursor.execute(query)

# i=0
# for row in cursor:
#     data.loc[i]=row;i+=1

#pickle.dump(data,open("euipo_en_data.p","wb"))

In [7]:
data=pickle.load(open("euipo_en_data.p","rb"))

In [8]:
# count_vec=CountVectorizer(tokenizer=word_tokenize)
# count_vec.fit_transform(data.loc[:,"text_data"])

# pickle.dump(count_vec,open("euipo_en_count_vec.p","wb"))
# pickle.dump(count_vec.vocabulary_,open("euipo_en_vocab.p","wb"))
count_vec=CountVectorizer(tokenizer=word_tokenize,vocabulary=pickle.load(open("euipo_en_vocab.p","rb")))

In [7]:
context_len=3
tokens=[word for word in word_tokenize(data.loc[0,"text_data"])]
tokens_SS=["START"]*context_len+tokens+["STOP"]*context_len

In [8]:
grams=[gram for gram in nltk.ngrams(tokens_SS,context_len*2+1)]

In [9]:
# for token,gram in zip(tokens[context_len:],grams):
#     if token!=gram[context_len]:
#         print token,gram[context_len]

In [10]:
features_df=pd.DataFrame(index=range(len(tokens)),columns=["context_l3","context_l2","context_l1","word","context_r1","context_r2","context_r3"])

features_df.loc[:]=grams[:]
features_df.head()

,context_l3,context_l2,context_l1,word,context_r1,context_r2,context_r3
0,START,START,START,OFFICE,FOR,HARMONIZATION,IN
1,START,START,OFFICE,FOR,HARMONIZATION,IN,THE
2,START,OFFICE,FOR,HARMONIZATION,IN,THE,INTERNAL
3,OFFICE,FOR,HARMONIZATION,IN,THE,INTERNAL,MARKET
4,FOR,HARMONIZATION,IN,THE,INTERNAL,MARKET,(


In [11]:
grams_list=[unicode(" ".join(gram)) for gram in grams]
context_features=count_vec.transform(grams_list)

In [12]:
word_features=[]
for word in tokens_SS:
    word_features.append([int(word.isalnum()),int(word.isalpha()),int(word.islower()),int(word.isupper())])

word_features_grams=nltk.ngrams(word_features,context_len*2+1)
word_features_array=np.ndarray.flatten(np.asarray(list(word_features_grams))[:]).reshape((len(tokens),4*(context_len*2+1)))

In [13]:
features_df_2=pd.DataFrame(index=range(len(tokens)),columns=["context_l3_alnum","context_l3_alpha","context_l3_lower","context_l3_upper","context_l2_alnum","context_l2_alpha","context_l2_lower","context_l2_upper","context_l1_alnum","context_l1_alpha","context_l1_lower","context_l1_upper","word_alnum","word_alpha","word_lower","word_upper","context_r1_alnum","context_r1_alpha","context_r1_lower","context_r1_upper","context_r2_alnum","context_r2_alpha","context_r2_lower","context_r2_upper","context_r3_alnum","context_r3_alpha","context_r3_lower","context_r3_upper"])
features_df_2.loc[:]=word_features_array

In [24]:
#print word_features_array.shape
print context_features.shape
print features_df.shape
print features_df_2.shape

(3428, 217174)
(3428, 7)
(3428, 28)


In [15]:
judges_df=pd.read_excel(open("training_set.xlsx","rb"),parse_cols="E,F,G")

temp_df=pd.DataFrame(columns=["judge_name_1","judge_name_2","judge_name_3"])

frames=[data,temp_df]
full_data=pd.concat(frames)

full_data.loc[0:len(judges_df),["judge_name_1","judge_name_2","judge_name_3"]]=judges_df.loc[0:len(judges_df),["judge_name_1","judge_name_2","judge_name_3"]]

In [16]:
label=np.zeros((len(tokens)))
for token in word_tokenize(full_data.judge_name_1[0]):
    label[tokens.index(token)]=1

In [26]:
temp_df=pd.DataFrame(columns=["Context_Features","Word_Features","Labels"])
frames=[full_data,temp_df]
labelled_data=pd.concat(frames)
